# Load model

In [1]:
import sys
sys.path.insert(0, '../')

import molecule_vae

grammar_weights = "../weights/GrammarVAE_L128.hdf5"
grammar_model = molecule_vae.GrammarModel(grammar_weights, latent_rep_size=128)

two_tower_grammar_weights = "../weights/Two_tower_GrammarVAE_L128.hdf5"
two_tower_grammar_model = molecule_vae.GrammarModel(two_tower_grammar_weights, latent_rep_size=128, two_tower=True)

character_weights = "../weights/CharVAE_L128.hdf5"
character_model = molecule_vae.CharacterModel(character_weights, latent_rep_size=128)

two_tower_character_weights = "../weights/Two_tower_CharVAE_L128.hdf5"
two_tower_character_model = molecule_vae.CharacterModel(two_tower_character_weights, latent_rep_size=128, two_tower=True)

Using TensorFlow backend.
C:\Users\jonat\Anaconda3\envs\multiview_chemvae\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jonat\Anaconda3\envs\multiview_chemvae\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jonat\Anaconda3\envs\multiview_chemvae\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\jo

# Clean data

In [2]:
import h5py
import numpy as np

grammar_data_path = '../grammar_dataset.h5'
features_data_path = '../features_dataset.h5'
char_data_path = '../char_dataset.h5'

#144338 158009
h5f = h5py.File(grammar_data_path, 'r')
grammar_data = h5f['data'][:]
h5f.close()
grammar_data=np.delete(grammar_data, [144338, 158009], axis=0) 

h5f = h5py.File(char_data_path, 'r')
char_data = h5f['data'][:]
h5f.close()
char_data=np.delete(char_data, [144338, 158009], axis=0) 

h5f = h5py.File(features_data_path, 'r')
rdkit_features = h5f['rdkit2d_normalized'][:]
h5f.close()
rdkit_features=np.expand_dims(rdkit_features, axis=2)
rdkit_features=np.delete(rdkit_features, [144338, 158009], axis=0)

# Calculate reconstruction accuracy

In [3]:
from tqdm import tqdm
# from sklearn.metrics import accuracy_score

charlist = ['C', '(', ')', 'c', '1', '2', 'o', '=', 'O', 'N', '3', 'F', '[',
            '@', 'H', ']', 'n', '-', '#', 'S', 'l', '+', 's', 'B', 'r', '/',
            '4', '\\', '5', '6', '7', 'I', 'P', '8', ' ']
_char_index = {}
for ix, char in enumerate(charlist):
    _char_index[char] = ix

def encode(smiles):
    """ Encode a list of smiles strings into the latent space """
    indices = [np.array([_char_index[c] for c in entry], dtype=int) for entry in smiles]
    one_hot = np.zeros((len(indices), 120, len(charlist)), dtype=np.float32)
    for i in range(len(indices)):
        num_productions = len(indices[i])
        one_hot[i][np.arange(num_productions),indices[i]] = 1.
        one_hot[i][np.arange(num_productions, 120),-1] = 1.
    return one_hot

def reconstruction(model, structures_data, features_data=None, grammar=False):
    
    avg_reconstructions = []

    for i in tqdm(range(structures_data.shape[0])):
        num_correct = 0
        structure_one_hot = structures_data[i]
        repeated_one_hot=np.array([structure_one_hot for i in range(10)])

        if features_data is not None:
            features = features_data[i]
            repeated_features = np.array([features for i in range(10)])
            encodings = model.vae.encoder.predict([repeated_one_hot, repeated_features])
            repeat_encodings = np.tile(encodings, (10,1))
            out, features = model.vae.decoder.predict(repeat_encodings)
        else:
            encodings = model.vae.encoder.predict(repeated_one_hot)
            repeat_encodings = np.tile(encodings, (10,1))
            out = model.vae.decoder.predict(repeat_encodings)

        if grammar:
            X_hat = model._sample_using_masks(out)
        else:
            noise = np.random.gumbel(size=out.shape)
            sampled_chars = np.argmax(np.log(out) + noise, axis=-1)
            char_matrix = np.array(charlist)[np.array(sampled_chars, dtype=int)]
            s=[''.join(ch).strip() for ch in char_matrix]
            X_hat=encode(s)

       
        for structure in X_hat:
            if np.array_equal(structure, structure_one_hot):
                num_correct+=1

        avg_reconstructions.append(num_correct/100)
        print(np.average(avg_reconstructions), np.std(avg_reconstructions))

    return np.average(avg_reconstructions)

In [4]:
# reconstruction (character_model, char_data[:5000], grammar=False)
# reconstruction (two_tower_character_model, char_data[:5000], rdkit_features[:5000], grammar=False)

# reconstruction (grammar_model, grammar_data[:5000], grammar=True)
reconstruction (two_tower_grammar_model, grammar_data[:5000], rdkit_features[:5000], grammar=True)

  0%|          | 0/5000 [00:00<?, ?it/s]WARNING:tensorflow:From C:\Users\jonat\Anaconda3\envs\multiview_chemvae\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

  0%|          | 15/5000 [01:59<11:03:07,  7.98s/it]


KeyboardInterrupt: 